# Download variables for TC tracking with UZ algorithm and TempestExtremes software
By Stella Bourdin

In [12]:
import os, intake, datetime
import xarray as xr
import numpy as np
from tqdm import tqdm

In [13]:
# Tracking parameters: Select zoom level, variables and levels
zoom = 8 # we select zoom 8 (~25km) as a reasonnable resolution for TC tracking

In [3]:
# Load catalog
cat = intake.open_catalog('https://digital-earths-global-hackathon.github.io/catalog/catalog.yaml')['UK']
# List available simulations
list(cat.keys())

['CERES_EBAF',
 'ERA5',
 'IR_IMERG',
 'JRA3Q',
 'MERRA2',
 'arp-gem-1p3km',
 'arp-gem-2p6km',
 'casesm2_10km_nocumulus',
 'icon_d3hp003',
 'icon_d3hp003aug',
 'icon_d3hp003feb',
 'icon_ngc4008',
 'ifs_tco3999-ng5_deepoff',
 'ifs_tco3999-ng5_rcbmf',
 'ifs_tco3999-ng5_rcbmf_cf',
 'ifs_tco3999_rcbmf',
 'nicam_220m_test',
 'nicam_gl11',
 'scream-dkrz',
 'tracking-d3hp003',
 'um_Africa_km4p4_RAL3P3_n1280_GAL9_nest',
 'um_CTC_km4p4_RAL3P3_n1280_GAL9_nest',
 'um_SAmer_km4p4_RAL3P3_n1280_GAL9_nest',
 'um_SEA_km4p4_RAL3P3_n1280_GAL9_nest',
 'um_glm_n1280_CoMA9_TBv1p2',
 'um_glm_n1280_GAL9',
 'um_glm_n2560_RAL3p3']

In [25]:
# Define the run you want to track, run-specific info + working directories
run='nicam_gl11'
## run-specific info
time_name_2D='PT1H'
time_name_3D='PT6H' #'PT3H'
level_name="lev" #"pressure"
level_in_hPa=True #True
orog_available=False #True

scr_dir = '/work/scratch-nopw2/sbourdin/'
run_dir = os.path.join(scr_dir,run)
if not os.path.isdir(run_dir):
    os.makedirs(run_dir)
output_dir = os.path.join(run_dir,'data_healpix/')
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

In [26]:
# Load data
ds_2d = cat[run](zoom=zoom, time=time_name_2D).to_dask()
ds_3d = cat[run](zoom=zoom, time=time_name_3D).to_dask()

/home/users/sbourdin/.conda/envs/hackathon/lib/python3.12/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),
/home/users/sbourdin/.conda/envs/hackathon/lib/python3.12/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),


In [27]:
# Select levels
if not level_in_hPa:
    ds_3d[level_name] = ds_3d[level_name] / 100
ds_3d = ds_3d.rename({level_name:"level"})

In [28]:
# Select 6-hourly
ds_2d = ds_2d.isel(time=(ds_2d.time.dt.hour % 6 == 0))
ds_3d = ds_3d.isel(time=(ds_3d.time.dt.hour % 6 == 0))

In [29]:
# Extract zg500 & zg250
ds_2d["zg500"] = ds_3d.zg.sel(level = 500)
ds_2d["zg250"] = ds_3d.zg.sel(level = 250)

In [30]:
# Merge and subset variables
variables = ['psl','uas','vas', 'zg500','zg250', 'orog']
if not orog_available:
    orog = xr.open_dataset(scr_dir+'orog/orog_zoom_'+str(zoom)+'.nc').orog.squeeze().drop_vars("crs")
    ds_2d = ds_2d.merge(orog)
ds = ds_2d[variables]

In [31]:
if "bounds" in list(ds.attrs.keys()):
    del ds.attrs["bounds"]
if "regional" in list(ds.attrs.keys()):
    del ds.attrs["regional"]

In [32]:
# Save one file per month
mth_list = np.unique(ds_2d.time.astype(str).str.slice(0,7))
for mth in tqdm(mth_list):
    fname = output_dir+mth+".nc"
    if not os.path.exists(fname):
        ds.sel(time = mth).to_netcdf(output_dir+mth+".nc")
    else:
        print(mth, "File already exists")

100%|██████████| 13/13 [10:27<00:00, 48.29s/it]


In [21]:
#rm /work/scratch-nopw2/sbourdin/arp-gem-2p6km/data_healpix/*